## Project Overview

This notebook performs data wrangling for the Demand Forecasting and Inventory Optimization project.

Goal:
Prepare clean and structured sales data for future analysis and machine learning modeling.

Dataset:
- M5 Forecasting Accuracy (Kaggle)


### Data Acquisition

In [3]:
import pandas as pd
import numpy as np

# Load datasets
sales = pd.read_csv("sales_train_validation.csv")
calendar = pd.read_csv("calendar.csv")
prices = pd.read_csv("sell_prices.csv")

### Inspect dataset sizes

In [4]:
print(sales.shape)
print(calendar.shape)
print(prices.shape)

(30490, 1919)
(1969, 14)
(6841121, 4)


In [5]:
sales.head()
sales.info()
sales.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.070220,1.041292,0.780026,0.833454,0.627944,0.958052,0.918662,1.244080,1.073663,0.838701,...,1.370581,1.586159,1.693670,1.248245,1.232207,1.159167,1.149000,1.328862,1.605838,1.633158
std,5.126689,5.365468,3.667454,4.415141,3.379344,4.785947,5.059495,6.617729,5.917204,4.206199,...,3.740017,4.097191,4.359809,3.276925,3.125471,2.876026,2.950364,3.358012,4.089422,3.812248
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
max,360.000000,436.000000,207.000000,323.000000,296.000000,314.000000,316.000000,370.000000,385.000000,353.000000,...,129.000000,160.000000,204.000000,98.000000,100.000000,88.000000,77.000000,141.000000,171.000000,130.000000


### Data Cleaning

In [6]:
sales.isnull().sum().sort_values(ascending=False)
calendar.isnull().sum()
prices.isnull().sum()

store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64

Missing values were inspected across datasets.
Missing price values were forward-filled because prices typically change gradually over time.

### Duplicate Records

In [17]:
print(sales.duplicated().sum())
print(prices.duplicated().sum())

prices.drop_duplicates(inplace=True)

# Drop duplicates if any (safe)
sales = sales.drop_duplicates()
calendar = calendar.drop_duplicates()
prices = prices.drop_duplicates()

0
0


Checked duplicates in all datasets and removed them if present to prevent double-counting during merges and data leakage during modeling.

### Data Type Fixes

In [8]:
calendar["date"] = pd.to_datetime(calendar["date"])
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int64         
 4   month         1969 non-null   int64         
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  162 non-null    object        
 8   event_type_1  162 non-null    object        
 9   event_name_2  5 non-null      object        
 10  event_type_2  5 non-null      object        
 11  snap_CA       1969 non-null   int64         
 12  snap_TX       1969 non-null   int64         
 13  snap_WI       1969 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 215.5+ KB


### Outlier Detection

In [10]:
sales.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.070220,1.041292,0.780026,0.833454,0.627944,0.958052,0.918662,1.244080,1.073663,0.838701,...,1.370581,1.586159,1.693670,1.248245,1.232207,1.159167,1.149000,1.328862,1.605838,1.633158
std,5.126689,5.365468,3.667454,4.415141,3.379344,4.785947,5.059495,6.617729,5.917204,4.206199,...,3.740017,4.097191,4.359809,3.276925,3.125471,2.876026,2.950364,3.358012,4.089422,3.812248
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
max,360.000000,436.000000,207.000000,323.000000,296.000000,314.000000,316.000000,370.000000,385.000000,353.000000,...,129.000000,160.000000,204.000000,98.000000,100.000000,88.000000,77.000000,141.000000,171.000000,130.000000


Extreme values represent real demand spikes during promotions or holidays and were kept to preserve business behavior.

### Data Reshaping

In [19]:
id_cols = ['id','item_id','dept_id','cat_id','store_id','state_id']

sales_long = sales.melt(
    id_vars=id_cols,
    var_name='d',
    value_name='sales'
)

print(sales_long.shape)
sales_long.head()

(58327370, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


### Merge calendar (adds real date + events)

In [20]:
df = sales_long.merge(calendar, on="d", how="left")

print(df.shape)
df[['id','d','date','sales','wm_yr_wk']].head()


(58327370, 21)


,id,d,date,sales,wm_yr_wk
0,HOBBIES_1_001_CA_1_validation,d_1,2011-01-29,0,11101
1,HOBBIES_1_002_CA_1_validation,d_1,2011-01-29,0,11101
2,HOBBIES_1_003_CA_1_validation,d_1,2011-01-29,0,11101
3,HOBBIES_1_004_CA_1_validation,d_1,2011-01-29,0,11101
4,HOBBIES_1_005_CA_1_validation,d_1,2011-01-29,0,11101


### Validation Check

In [22]:
df.isnull().sum().sort_values(ascending=False).head(15)

event_type_2    58205410
event_name_2    58205410
event_type_1    53631910
event_name_1    53631910
id                     0
wday                   0
snap_TX                0
snap_CA                0
year                   0
month                  0
weekday                0
item_id                0
wm_yr_wk               0
date                   0
sales                  0
dtype: int64

### Merge prices

In [26]:
df = df.merge(
    prices,
    on=["store_id", "item_id", "wm_yr_wk"],
    how="left"
)

print(df.shape)
df[['id','date','sales','sell_price']].head()

(58327370, 24)


,id,date,sales,sell_price
0,HOBBIES_1_001_CA_1_validation,2011-01-29,0,NaN
1,HOBBIES_1_002_CA_1_validation,2011-01-29,0,NaN
2,HOBBIES_1_003_CA_1_validation,2011-01-29,0,NaN
3,HOBBIES_1_004_CA_1_validation,2011-01-29,0,NaN
4,HOBBIES_1_005_CA_1_validation,2011-01-29,0,NaN


In [27]:
df.isnull().sum().sort_values(ascending=False).head(20)

event_type_2    58205410
event_name_2    58205410
event_type_1    53631910
event_name_1    53631910
sell_price      12299413
sell_price_y    12299413
sell_price_x    12299413
item_id                0
snap_WI                0
snap_TX                0
snap_CA                0
year                   0
id                     0
wday                   0
weekday                0
wm_yr_wk               0
date                   0
sales                  0
d                      0
state_id               0
dtype: int64

### Handeling missing sell_price

In [28]:
df = df.sort_values(["store_id","item_id","date"])

df["sell_price"] = (
    df.groupby(["store_id","item_id"])["sell_price"]
      .ffill()
      .bfill()
)

# If anything still missing:
df["sell_price"] = df["sell_price"].fillna(0)

#### Missing Values
The raw `prices` table contains no missing values, but missing `sell_price` can appear after merging when an item-store-week combination has no listed price.  
To preserve the time series continuity, prices were forward-filled and backfilled within each item-store group, then any remaining missing values were set to 0 (rare cases).

### Outlier check
In demand forecasting, spikes are often real (promotions/holidays).

In [29]:
df["sales"].describe(percentiles=[0.95, 0.99, 0.999])

count    5.832737e+07
mean     1.126322e+00
std      3.873108e+00
min      0.000000e+00
50%      0.000000e+00
95%      5.000000e+00
99%      1.500000e+01
99.9%    4.700000e+01
max      7.630000e+02
Name: sales, dtype: float64

### Basic feature prep

In [31]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["day_of_week"] = df["date"].dt.dayofweek
df["week_of_year"] = df["date"].dt.isocalendar().week.astype(int)

In [32]:
df.info()
df.isnull().sum().sort_values(ascending=False).head(20)
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 58327370 entries, 1612 to 58325932
Data columns (total 27 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       object        
 2   dept_id       object        
 3   cat_id        object        
 4   store_id      object        
 5   state_id      object        
 6   d             object        
 7   sales         int64         
 8   date          datetime64[ns]
 9   wm_yr_wk      int64         
 10  weekday       object        
 11  wday          int64         
 12  month         int32         
 13  year          int32         
 14  event_name_1  object        
 15  event_type_1  object        
 16  event_name_2  object        
 17  event_type_2  object        
 18  snap_CA       int64         
 19  snap_TX       int64         
 20  snap_WI       int64         
 21  sell_price_x  float64       
 22  sell_price_y  float64       
 23  sell_price    float64       
 24

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,event_type_2,snap_CA,snap_TX,snap_WI,sell_price_x,sell_price_y,sell_price,day,day_of_week,week_of_year
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,...,NaN,0,0,0,2.0,2.0,2.0,29,5,4
32102,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0,2011-01-30,11101,...,NaN,0,0,0,2.0,2.0,2.0,30,6,4
62592,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0,2011-01-31,11101,...,NaN,0,0,0,2.0,2.0,2.0,31,0,5
93082,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1,2011-02-01,11101,...,NaN,1,1,0,2.0,2.0,2.0,1,1,5
123572,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4,2011-02-02,11101,...,NaN,1,0,1,2.0,2.0,2.0,2,2,5


In [ ]:
df.to_csv("m5_clean_merged_long.csv", index=False)

## Data Wrangling Summary

Steps completed:
- Loaded M5 sales, calendar, and price datasets
- Inspected shapes, data types, and summary statistics
- Checked and removed duplicates where applicable
- Reshaped sales data from wide format (d_1…d_1913) to long format
- Merged calendar data to attach real dates and event metadata
- Merged sell prices using store_id, item_id, and wm_yr_wk
- Handled missing sell_price values created during merging using group-wise forward/back fill
- Performed outlier inspection and retained demand spikes as legitimate business behavior
- Created basic time features to support later EDA and modeling

Result:
A cleaned, merged dataset ready for exploratory analysis and forecasting models.

### Business Interpretation

Demand spikes may reflect promotions or seasonality.
Preserving these patterns is important for accurate forecasting models.